<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao26_NMT_RNN_Benchmark_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# coding: utf-8
# ==============================================================================
# SCRIPT PYTHON PARA GOOGLE COLAB
#
# TAREFA: Avaliação de Benchmark de Tradução Automática (NMT)
# MODELO: RNN (Seq2Seq com LSTM)
# MÉTRICA: BLEU Score
#
# INSTRUÇÕES:
# 1. Abra um novo notebook no Google Colab (https://colab.research.google.com/)
# 2. Copie e cole o conteúdo deste arquivo, célula por célula, e execute.
#
# As seções marcadas com "# --- CÉLULA X ---" devem ser coladas
# em células separadas do Colab.
# ==============================================================================

# --- CÉLULA 1: Instalação e Imports ---
# (Instala o NLTK para a métrica BLEU e baixa os pacotes necessários)

!pip install nltk
import nltk
nltk.download('punkt')

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

import numpy as np
import os
import re
import zipfile
import urllib.request

print(f"TensorFlow Versão: {tf.__version__}")

# --- CÉLULA 2: Parâmetros e Download dos Dados ---
# (Vamos usar um dataset Português-Inglês do http://www.manythings.org/anki/)

#@title Parâmetros do Modelo e Dados
NUM_EXEMPLOS = 30000  # Quantidade de frases para treinar. Reduza se o Colab for lento.
BATCH_SIZE = 64
EPOCHS = 30
LATENT_DIM = 256  # Dimensão da camada LSTM

# URL do dataset
DATA_URL = "https://github.com/jsansao/transformers_pt/raw/refs/heads/main/por-eng.zip"
DATA_PATH = "por-eng.zip"
EXTRACT_PATH = "por-eng"

# Download
if not os.path.exists(DATA_PATH):
    urllib.request.urlretrieve(DATA_URL, DATA_PATH)
    print("Dataset baixado.")

# Extração
if not os.path.exists(EXTRACT_PATH):
    os.makedirs(EXTRACT_PATH)
with zipfile.ZipFile(DATA_PATH, "r") as zip_ref:
    zip_ref.extractall(EXTRACT_PATH)
    print("Dataset extraído.")

DATA_FILE = os.path.join(EXTRACT_PATH, "por.txt")
print(f"Arquivo de dados: {DATA_FILE}")


# --- CÉLULA 3: Carregamento e Pré-processamento ---

def preprocess_sentence(s):
    """
    Limpa e prepara uma única sentença.
    Adiciona tokens de [start] e [end].
    """
    s = s.lower().strip()
    # Adiciona espaço antes da pontuação para tokenização
    s = re.sub(r"([?.!,¿])", r" \1 ", s)
    s = re.sub(r'[" "]+', " ", s)
    # Substitui tudo exceto letras e pontuação básica
    s = re.sub(r"[^a-zA-Záéíóúâêîôûãõç?.!,¿]+", " ", s)
    s = s.strip()
    # Adiciona os tokens de início e fim
    s = '[start] ' + s + ' [end]'
    return s

def load_data(path, num_examples):
    """
    Lê o arquivo .txt e retorna pares de frases (Inglês, Português).
    """
    # O arquivo está no formato: Inglês \t Português \t Atribuição
    pairs = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f.readlines()[:num_examples]:
            parts = line.split('\t')
            # Usamos o Português como ENTRADA (input) e Inglês como SAÍDA (target)
            # Você pode inverter se preferir (target_lang, input_lang)
            input_text = preprocess_sentence(parts[1])  # Português
            target_text = preprocess_sentence(parts[0]) # Inglês
            pairs.append((input_text, target_text))

    return zip(*pairs)

# Carrega os dados
input_lang_raw, target_lang_raw = load_data(DATA_FILE, NUM_EXEMPLOS)
print("Exemplo de dados brutos:")
print(f"Input (PT): {input_lang_raw[0]}")
print(f"Target (EN): {target_lang_raw[0]}")


# --- CÉLULA 4: Tokenização e Padding ---
# (Transforma as palavras em números (índices) e preenche as sequências)

def tokenize(lang):
    """
    Cria um tokenizador Keras para um idioma.
    """
    # oov_token='<unk>' -> Palavras desconhecidas serão mapeadas para <unk>
    lang_tokenizer = Tokenizer(filters='', oov_token='<unk>')
    lang_tokenizer.fit_on_texts(lang)
    return lang_tokenizer

def texts_to_sequences(tokenizer, texts):
    """
    Converte textos em sequências de inteiros.
    """
    return tokenizer.texts_to_sequences(texts)

# Tokeniza o idioma de ENTRADA (Português)
input_tokenizer = tokenize(input_lang_raw)
input_tensor = texts_to_sequences(input_tokenizer, input_lang_raw)
input_tensor = pad_sequences(input_tensor, padding='post')

# Tokeniza o idioma ALVO (Inglês)
target_tokenizer = tokenize(target_lang_raw)
target_tensor = texts_to_sequences(target_tokenizer, target_lang_raw)
target_tensor = pad_sequences(target_tensor, padding='post')

# Índices (Word -> ID) e Índices Inversos (ID -> Word)
input_word_index = input_tokenizer.word_index
input_index_word = input_tokenizer.index_word
target_word_index = target_tokenizer.word_index
target_index_word = target_tokenizer.index_word

# Tamanho dos vocabulários (adiciona 1 para o token 0 de padding)
VOCAB_SIZE_INPUT = len(input_word_index) + 1
VOCAB_SIZE_TARGET = len(target_word_index) + 1

# Tamanho máximo das sequências
MAX_LEN_INPUT = input_tensor.shape[1]
MAX_LEN_TARGET = target_tensor.shape[1]

print("\n--- Estatísticas dos Dados ---")
print(f"Frases de entrada (PT): {len(input_tensor)}")
print(f"Frases de saída (EN): {len(target_tensor)}")
print(f"Tamanho Vocabulário PT: {VOCAB_SIZE_INPUT}")
print(f"Tamanho Vocabulário EN: {VOCAB_SIZE_TARGET}")
print(f"Tamanho Máx. Sequência PT: {MAX_LEN_INPUT}")
print(f"Tamanho Máx. Sequência EN: {MAX_LEN_TARGET}")

# --- CÉLULA 5: Preparação dos Dados para Treinamento (Teacher Forcing) ---
# (O modelo Seq2Seq precisa de 3 partes: entrada do encoder, entrada do decoder e saída do decoder)

# 1. encoder_input_data: A frase em Português
encoder_input_data = input_tensor

# 2. decoder_input_data: A frase em Inglês, "deslocada"
#    (Ex: "[start] a cat sat [end]" -> "[start] a cat sat")
#    Isso é o "teacher forcing".
decoder_input_data = target_tensor[:, :-1]

# 3. decoder_target_data: A frase em Inglês, "deslocada" para o alvo
#    (Ex: "[start] a cat sat [end]" -> "a cat sat [end]")
#    O modelo tenta prever esta sequência.
decoder_target_data = target_tensor[:, 1:]

# Ajusta a forma do target para a loss function
decoder_target_data = np.expand_dims(decoder_target_data, -1)

print("\n--- Formas dos Dados de Treinamento ---")
print(f"Encoder Input (PT): {encoder_input_data.shape}")
print(f"Decoder Input (EN - Teacher Forcing): {decoder_input_data.shape}")
print(f"Decoder Target (EN - Labels): {decoder_target_data.shape}")

# Cria um tf.data.Dataset para eficiência
BUFFER_SIZE = len(input_tensor)
# Modify the dataset creation to yield inputs as a tuple in the correct order
dataset = tf.data.Dataset.from_tensor_slices(((encoder_input_data, decoder_input_data), decoder_target_data))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Separa 10% para validação/benchmark
validation_size = int(0.1 * len(input_tensor))
train_size = len(input_tensor) - validation_size

# Pega os dados de validação ANTES de embaralhar o dataset
# (Usaremos esses dados crus para o benchmark BLEU)
validation_inputs_raw = input_lang_raw[train_size:]
validation_targets_raw = target_lang_raw[train_size:]

# Pega os tensores de validação
validation_encoder_input = encoder_input_data[train_size:]
# (Não precisamos dos outros para a inferência)

print(f"\nTamanho Treino: {train_size}")
print(f"Tamanho Validação (Benchmark): {validation_size}")

# --- CÉLULA 6: Definição do Modelo Seq2Seq (RNN/LSTM) ---
# (Este é o modelo de *treinamento* que usa teacher forcing)

# --- ENCODER (Codificador) ---
# Recebe a sequência de entrada (PT)
encoder_inputs = Input(shape=(MAX_LEN_INPUT,), name="encoder_input")
# Camada de Embedding (transforma IDs em vetores densos)
encoder_embedding = Embedding(VOCAB_SIZE_INPUT, LATENT_DIM, name="encoder_embedding")(encoder_inputs)
# Camada LSTM (o "RNN")
# return_state=True faz o LSTM retornar seu estado interno (o "vetor de pensamento")
encoder_lstm = LSTM(LATENT_DIM, return_state=True, name="encoder_lstm")
# Não precisamos das saídas (outputs) do encoder, apenas dos estados
_, state_h, state_c = encoder_lstm(encoder_embedding)
# Agrupamos os estados (hidden state e cell state)
encoder_states = [state_h, state_c]

# --- DECODER (Decodificador) ---
# Recebe a sequência alvo (EN) para o teacher forcing
decoder_inputs = Input(shape=(MAX_LEN_TARGET - 1,), name="decoder_input") # (Lembre-se que removemos o último token)
# Camada de Embedding (compartilhar pesos é avançado, aqui usamos separado)
decoder_embedding_layer = Embedding(VOCAB_SIZE_TARGET, LATENT_DIM, name="decoder_embedding")
decoder_embedding = decoder_embedding_layer(decoder_inputs)
# Camada LSTM do Decoder
# return_sequences=True faz o LSTM retornar a sequência completa de saídas
decoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True, name="decoder_lstm")
# IMPORTANTE: Inicializamos o estado do Decoder com os estados finais do Encoder
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
# Camada Densa final para classificação sobre o vocabulário alvo
decoder_dense = Dense(VOCAB_SIZE_TARGET, activation='softmax', name="decoder_dense")
decoder_outputs = decoder_dense(decoder_outputs)

# --- MODELO COMPLETO (Treinamento) ---
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compila o modelo
# Usamos 'sparse_categorical_crossentropy' porque nossos alvos (decoder_target_data) são inteiros, não one-hot.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


# --- CÉLULA 7: Treinamento do Modelo ---
# (Isso pode levar alguns minutos no Colab)

print("\nIniciando o treinamento...")
history = model.fit(dataset.take(train_size // BATCH_SIZE),
                    epochs=EPOCHS,
                    validation_data=dataset.skip(train_size // BATCH_SIZE),
                    verbose=1)
print("Treinamento concluído.")


# --- CÉLULA 8: Definição dos Modelos de Inferência (Tradução) ---
# (Para traduzir, precisamos de um modelo diferente que gere palavra por palavra)

# O processo de tradução (inferência) é diferente do treinamento.
# 1. Codificamos a frase de entrada (PT) e obtemos os estados [state_h, state_c].
# 2. Alimentamos o decoder com o token [start] e os estados do encoder.
# 3. O decoder prevê a próxima palavra (ex: "a").
# 4. Alimentamos o decoder com a palavra "a" e os *novos* estados internos do decoder.
# 5. Repetimos até o decoder prever [end] ou atingir o limite de tamanho.

# --- 1. Modelo Encoder de Inferência ---
# (Pega a entrada PT e retorna os estados)
encoder_model = Model(encoder_inputs, encoder_states)

# --- 2. Modelo Decoder de Inferência ---
# (Pega o token anterior e os estados anteriores, e retorna o novo token e novos estados)

# Inputs de estado para o decoder
decoder_state_h_input = Input(shape=(LATENT_DIM,))
decoder_state_c_input = Input(shape=(LATENT_DIM,))
decoder_states_inputs = [decoder_state_h_input, decoder_state_c_input]

# Input para o token (agora com tamanho 1)
decoder_single_input = Input(shape=(1,))

# Reutilizamos as camadas de embedding e LSTM do modelo treinado
decoder_embedding_inference = decoder_embedding_layer(decoder_single_input)
decoder_outputs_inference, state_h_inf, state_c_inf = decoder_lstm(
    decoder_embedding_inference, initial_state=decoder_states_inputs)

decoder_states_inference = [state_h_inf, state_c_inf]
decoder_outputs_inference = decoder_dense(decoder_outputs_inference)

# O modelo final do decoder
decoder_model = Model(
    [decoder_single_input] + decoder_states_inputs,
    [decoder_outputs_inference] + decoder_states_inference
)

print("\nModelos de inferência (encoder e decoder) criados.")
encoder_model.summary()
decoder_model.summary()


# --- CÉLULA 9: Função de Tradução (Inferência) ---

def translate_sentence(input_seq):
    """
    Traduz uma única sequência (pré-processada) usando os modelos de inferência.
    """
    # 1. Obtém os estados do encoder
    states_value = encoder_model.predict(input_seq, verbose=0)

    # 2. Prepara o input inicial do decoder (o token [start])
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_word_index['[start]']

    stop_condition = False
    decoded_sentence = []

    while not stop_condition:
        # 3. Prediz a próxima palavra
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)

        # 4. Obtém o ID da palavra prevista (a mais provável)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = target_index_word[sampled_token_index]

        # 5. Adiciona a palavra à sentença (if not [end])
        if sampled_word == '[end]':
            stop_condition = True
        else:
            decoded_sentence.append(sampled_word)

        # 6. Check for stop condition
        if stop_condition or len(decoded_sentence) > MAX_LEN_TARGET:
            stop_condition = True

        # 7. Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 8. Update states
        states_value = [h, c]

    return ' '.join(decoded_sentence)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TensorFlow Versão: 2.19.0
Dataset extraído.
Arquivo de dados: por-eng/por.txt
Exemplo de dados brutos:
Input (PT): [start] vai . [end]
Target (EN): [start] go . [end]

--- Estatísticas dos Dados ---
Frases de entrada (PT): 30000
Frases de saída (EN): 30000
Tamanho Vocabulário PT: 7206
Tamanho Vocabulário EN: 4168
Tamanho Máx. Sequência PT: 14
Tamanho Máx. Sequência EN: 10

--- Formas dos Dados de Treinamento ---
Encoder Input (PT): (30000, 14)
Decoder Input (EN - Teacher Forcing): (30000, 9)
Decoder Target (EN - Labels): (30000, 9, 1)

Tamanho Treino: 27000
Tamanho Validação (Benchmark): 3000


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input       │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, 14, 256)   │  1,844,736 │ encoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, 9, 256)    │  1,067,008 │ decoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 256),     │    525,312 │ encoder_embeddin… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, 9, 256),  │    525,312 │ decoder_embeddin… │
│                     │ (None, 256),      │            │ encoder_lstm[0][… │
│                     │ (None, 256)]      │            │ encoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, 9, 4168)   │  1,071,176 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,033,544 (19.20 MB)

...

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, 1, 256)    │  1,067,008 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, 1, 256),  │    525,312 │ decoder_embeddin… │
│                     │ (None, 256),      │            │ input_layer_3[0]… │
│                     │ (None, 256)]      │            │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, 1, 4168)   │  1,071,176 │ decoder_lstm[1][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,663,496 (10.16 MB)

 Trainable params: 2,663,496 (10.16 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# --- CÉLULA 10: Avaliação do Benchmark (BLEU Score) ---
# (Calculamos a métrica BLEU no conjunto de validação)

print("\n--- Iniciando Avaliação do Benchmark (BLEU Score) ---")

references = [] # The real translations (gold)
predictions = [] # The model's translations (hypotheses)

# Iterate over the validation set (benchmark)
# (Use .iloc to ensure we are getting the correct data)

for i in range(len(validation_encoder_input)):

    input_seq = validation_encoder_input[i:i+1] # Get the input sequence

    # Translate the sequence
    predicted_sentence = translate_sentence(input_seq)

    # Get the real target sentence (reference)
    # Remove [start] and [end] from the reference for BLEU calculation
    reference_sentence_raw = validation_targets_raw[i]
    reference_sentence = reference_sentence_raw.replace('[start] ', '').replace(' [end]', '')

    # NLTK expects lists of words
    references.append([reference_sentence.split()]) # BLEU can have multiple refs, hence [[]]
    predictions.append(predicted_sentence.split())

    if i % 100 == 0:
        print(f"Evaluating {i}/{len(validation_encoder_input)}...")
        print(f"  Input (PT): {validation_inputs_raw[i]}")
        print(f"  Real (EN): {reference_sentence}")
        print(f"  Prev (EN): {predicted_sentence}")

# --- FINAL BLEU CALCULATION ---
# BLEU-1 (unigrams)
bleu_1 = corpus_bleu(references, predictions, weights=(1, 0, 0, 0))
# BLEU-4 (standard)
bleu_4 = corpus_bleu(references, predictions, weights=(0.25, 0.25, 0.25, 0.25))

print("\n--- BENCHMARK RESULT ---")
print(f"Total sentences evaluated: {len(references)}")
print(f"BLEU-1 Score (Unigrams): {bleu_1 * 100:.2f}")
print(f"BLEU-4 Score (Corpus): {bleu_4 * 100:.2f}")



--- Iniciando Avaliação do Benchmark (BLEU Score) ---
Evaluating 0/3000...
  Input (PT): [start] tu és inflexível . [end]
  Real (EN): you re inflexible .
  Prev (EN): you re inflexible .
Evaluating 100/3000...
  Input (PT): [start] todos nós conhecemos tom . [end]
  Real (EN): all of us know tom .
  Prev (EN): we all know tom .
Evaluating 200/3000...
  Input (PT): [start] você é o médico ? [end]
  Real (EN): are you the doctor ?
  Prev (EN): are you the doctor ?
Evaluating 300/3000...
  Input (PT): [start] cheque aquele carro . [end]
  Real (EN): check out that car .
  Prev (EN): check out that car .
Evaluating 400/3000...
  Input (PT): [start] você tem o bastante ? [end]
  Real (EN): do you have enough ?
  Prev (EN): do you have enough ?
Evaluating 500/3000...
  Input (PT): [start] será que o gosto é bom ? [end]
  Real (EN): does it taste good ?
  Prev (EN): does it taste good ?
Evaluating 600/3000...
  Input (PT): [start] não escreva a tinta . [end]
  Real (EN): don t write in ink 

In [17]:

print("\n--- Manual Test ---")
# Pick a random sentence from the validation set
idx = np.random.randint(0, len(validation_encoder_input))

input_seq_test = validation_encoder_input[idx:idx+1]
input_raw_test = validation_inputs_raw[idx]
target_raw_test = validation_targets_raw[idx].replace('[start] ', '').replace(' [end]', '')
predicted_test = translate_sentence(input_seq_test)

print(f"Input Sentence (PT): {input_raw_test}")
print(f"Real Translation (EN):    {target_raw_test}")
print(f"Model Translation (EN): {predicted_test}")


--- Manual Test ---
Input Sentence (PT): [start] não comprem essa . [end]
Real Translation (EN):    don t buy that one .
Model Translation (EN): don t buy that one .


In [32]:
# --- CÉLULA 11: Função para Traduzir uma String Arbitrária ---
# (Usa as funções de pré-processamento e inferência definidas anteriormente)

def translate_string(input_string):
    """
    Traduz uma string de texto em Português para Inglês.
    """
    # 1. Pré-processa a string de entrada
    processed_string = preprocess_sentence(input_string)

    # 2. Converte a string processada para uma sequência de IDs
    # O tokenizer espera uma lista de strings
    input_seq = texts_to_sequences(input_tokenizer, [processed_string])
    # Garante o padding correto
    input_seq = pad_sequences(input_seq, maxlen=MAX_LEN_INPUT, padding='post')

    # 3. Chama a função de inferência para traduzir a sequência
    translated_sentence = translate_sentence(input_seq)

    return translated_sentence

# --- Exemplo de uso ---
test_string = "Olá, como você está?"
translated_string = translate_string(test_string)
print(f"\nOriginal (PT): {test_string}")
print(f"Traduzido (EN): {translated_string}")

test_string_2 = "Amostra grátis"
translated_string_2 = translate_string(test_string_2)
print(f"\nOriginal (PT): {test_string_2}")
print(f"Traduzido (EN): {translated_string_2}")


Original (PT): Olá, como você está?
Traduzido (EN): hello , how are you ?

Original (PT): Amostra grátis
Traduzido (EN): attendance is free .
